In [112]:
# Identify athletes meeting new Dec 24 OCTC selection rules: 

#1. Segregate into Male and Femal 
#2. For each gender perform the following: 
#a. Sort data by mapped eent, then perf scalar (higher the better)
#b. Identify tiers based on performance - Tier 1 is meets bronze medal mark for SEAG, Tier 2 is 2% and Tier 3 is 3.5%
#c. Check - if athlete met bronze or 2%/3.5% then delta_benchmark is zero or +, delta2% is + and delta 3.5% is +
#d. Top ranked athletes for each event are chosen. Max number of athletes for each event is 3, except for 100m/400m which is 6
#    This includes athletes on spex scholarship and potential
#e. The max for each tier is 2. Lower ranked athletes move down one tier.
#3. If athlete qualifies for more than one event the higher tier event is given
#4. Jump and throws junior program to be solved separately

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2565]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime
import icecream as ic
import dateutil.parser as parser 
import datacompy



from google.cloud import storage



Please note that you are missing the optional dependency: fugue. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: spark. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: snowflake. If you need to use this functionality it must be installed.


In [4136]:
# PRODUCTION ENVIRONMENT
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, TEAM, AGE, RANK AS COMPETITION_RANK, DIVISION, EVENT, DISTANCE, EVENT_CLASS, UNIQUE_ID, DOB, NATIONALITY, WIND, CATEGORY_EVENT, GENDER, COMPETITION, DATE, YEAR, REGION
FROM `saa-analytics.results.athlete_results_prod` 
WHERE RESULT!='NM' AND RESULT!='-' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL

"""

competitors = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [4137]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,23/4/97,,,None,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,1/8/19,nan,0.5,None,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,3/3/74,nan,,Throw,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,28/12/02,IND,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,9/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,6/9/10,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,20/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local


In [4138]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


competitors.to_csv('athletes_new_format_prod.csv', sep=',', encoding='utf-8-sig', index=False)

In [4139]:
competitors

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,DOB,NATIONALITY,WIND,CATEGORY_EVENT,GENDER,COMPETITION,DATE,YEAR,REGION
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,24/11/90,nan,,Throw,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,4/5/95,nan,,Jump,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,23/4/97,,,None,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,1/8/19,nan,0.5,None,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,3/3/74,nan,,Throw,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,28/12/02,IND,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,9/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,6/9/10,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,20/7/09,SIN,None,Relay,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local


In [4250]:
competitors[competitors['COMPETITION']=='National School Games']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month
1695,Tan Jun Ji,00:15.69,BG,None,102,C,100m,None,None,None,...,Male,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN
1696,Madhavan Pranav,01:03.21,BV,None,72,B,400m,None,None,None,...,Male,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN
1697,"Ko Weijun, Darius",01:00.16,BV,None,50,B,400m,None,None,None,...,Male,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN
1698,Jeremy Yap Yu Cai,01:02.34,BV,None,68,B,400m,None,None,None,...,Male,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN
1699,Rachael Lim Zhi Yao,01:20.22,BV,None,52,B,400m,None,None,None,...,Female,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,Teo Lin,00:23.98,SJI(INT),None,12,B,200m,None,None,None,...,Male,National School Games,11-04,2024,Local,04/11/24,NaT,NaT,NaN,NaN
6152,Faith Jia Yin Koh,01:02.44,SJI(INT),None,3,B,400m,None,None,None,...,Female,National School Games,13-04,2024,Local,04/13/24,NaT,NaT,NaN,NaN
6153,Caitlin Ng Shan Wen,00:27.35,SJI(INT),None,1,C,200m,None,None,None,...,Female,National School Games,15-04,2024,Local,04/15/24,NaT,NaT,NaN,NaN
6154,Arjun Mehta,1.55,SJI(INT),None,13,B,High Jump,None,None,None,...,Male,National School Games,19-04,2024,Local,04/19/24,NaT,NaT,NaN,NaN


In [4141]:
# Create mm/dd/yy column for date of event

import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

for i in range(len(competitors)):
        
    rowIndex = competitors.index[i]

    date = competitors.loc[rowIndex,'DATE']
    year = competitors.loc[rowIndex,'YEAR']    
    
    if 'to' in date or ' - ' in date:
        
        pos = re.search('to|\s\-\s', date)
                                                
        # Splice string to day and month

        split_pos_start=pos.start()+3

            
        final_date = date[split_pos_start:] # left string post splicing
        final_year = year[2:]
        
        event_date = final_date + '/' + final_year
        
        competitors.loc[rowIndex, 'event_date'] = event_date
        
    elif re.search('\w\-\w', date):
        
        if competitors.iloc[rowIndex, 15] == "National School Games":
        
            event_date = '04'+'/'+date[1:3] + '/' + year[2:]  # reverse order from dd/mm to mm/dd
        
            competitors.loc[rowIndex, 'event_date'] = event_date
        
        else:
            
            event_date = date + '-' + year[2:]
            
            competitors.loc[rowIndex, 'event_date'] = event_date

        
        

        
competitors['event_date'] = competitors['event_date'].astype(str)
competitors['event_date'] = competitors['event_date'].str.replace('\xa0', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\r', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.replace('\n', ' ', regex=True)
competitors['event_date'] = competitors['event_date'].str.strip()
       
    

In [4142]:
# Calculate number of days from today to event date

competitors['event_date_dt'] = pd.to_datetime(competitors['event_date'], format='%d%m%Y', errors='coerce')

competitors['delta_time']= datetime.datetime.now() - competitors['event_date_dt']

competitors['delta_time_conv'] = pd.to_numeric(competitors['delta_time'].dt.days, downcast='integer')

competitors['event_month'] = competitors['event_date_dt'].dt.month


In [4143]:
# Choose last 6 months or 12 months of results 

#competitors = competitors[(competitors['delta_time_conv']>=0) & (competitors['delta_time_conv']<=365)]

#competitors=competitors.reset_index(drop=True)

#start = datetime.datetime(2024, 10, 22)

#end = datetime.datetime(2025, 3, 31)

#start_date = np.datetime64(start)
#end_date = np.datetime64(end)


#mask = (competitors['event_date_dt'] >= start_date) & (competitors['event_date_dt'] <= end_date)
#athletes_selected = competitors.loc[mask]

In [4144]:
athletes_selected = competitors[(competitors['YEAR']=='2024')|(competitors['YEAR']=='2025')]

#athletes_selected = competitors[(competitors['YEAR']=='2025')]

In [4145]:
athletes_selected

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,GENDER,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,11/10/24,NaT,NaT,NaN,NaN
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,Male,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,11/10/24,NaT,NaT,NaN,NaN
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,Male,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local,9/7/24,NaT,NaT,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,Female,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,6/22/24,NaT,NaT,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,Male,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,6/22/24,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,NaN,NaN
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,NaN,NaN
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,NaN,NaN
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,Male,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,NaN,NaN


In [4251]:
athletes_selected[athletes_selected['COMPETITION']=='National School Games']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
1695,Tan Jun Ji,00:15.69,BG,None,102,C,100m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,
1696,Madhavan Pranav,01:03.21,BV,None,72,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,
1697,"Ko Weijun, Darius",01:00.16,BV,None,50,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,
1698,Jeremy Yap Yu Cai,01:02.34,BV,None,68,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,
1699,Rachael Lim Zhi Yao,01:20.22,BV,None,52,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,Teo Lin,00:23.98,SJI(INT),None,12,B,200m,None,None,None,...,National School Games,11-04,2024,Local,04/11/24,NaT,NaT,nan,nan,
6152,Faith Jia Yin Koh,01:02.44,SJI(INT),None,3,B,400m,None,None,None,...,National School Games,13-04,2024,Local,04/13/24,NaT,NaT,nan,nan,
6153,Caitlin Ng Shan Wen,00:27.35,SJI(INT),None,1,C,200m,None,None,None,...,National School Games,15-04,2024,Local,04/15/24,NaT,NaT,nan,nan,
6154,Arjun Mehta,1.55,SJI(INT),None,13,B,High Jump,None,None,None,...,National School Games,19-04,2024,Local,04/19/24,NaT,NaT,nan,nan,High Jump


In [4252]:
athletes_selected.to_csv('athletes_selected_prod.csv', encoding='utf-8')

In [4253]:
# Choose 2024/25 only

athletes = athletes_selected

In [4254]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,11/10/24,NaT,NaT,nan,nan,Hammer Throw
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,11/10/24,NaT,NaT,nan,nan,Pole Vault
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local,9/7/24,NaT,NaT,nan,nan,
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,6/22/24,NaT,NaT,nan,nan,
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,6/22/24,NaT,NaT,nan,nan,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,nan,nan,4 x 100m
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,nan,nan,4 x 100m
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,nan,nan,4 x 100m
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,nan,nan,4 x 100m


In [4255]:
athletes[athletes['NAME']=='Jun Jie Calvin Quek']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
1413,Jun Jie Calvin Quek,53.19,None,None,4,None,400m Hurdles,None,None,None,...,UAE Athletics Grand Prix,3-May,2024,International,3-May-24,NaT,NaT,nan,nan,400m Hurdles
1414,Jun Jie Calvin Quek,52.05,None,None,8,None,400m Hurdles,None,None,None,...,11th Kinami Michitaka Memorial Athletics Meet,12-May,2024,International,12-May-24,NaT,NaT,nan,nan,400m Hurdles
1516,Jun Jie Calvin Quek,51.8,None,None,1,None,400m Hurdles,None,None,None,...,Taiwan Athletics Open,1 - 2 June,2024,International,2 June/24,NaT,NaT,nan,nan,400m Hurdles
1544,Jun Jie Calvin Quek,51.79,None,None,2,None,400m Hurdles,None,None,None,...,XXXIII Qosanov Memorial,22 - 23 June,2024,International,23 June/24,NaT,NaT,nan,nan,400m Hurdles
1545,Jun Jie Calvin Quek,56.95,None,None,3,None,400m Hurdles,None,None,None,...,XXXIII Qosanov Memorial,22 - 23 June,2024,International,23 June/24,NaT,NaT,nan,nan,400m Hurdles
1689,Jun Jie Calvin Quek,52.18,None,None,3,None,400m Hurdles,None,None,None,...,Second Belt and Road Invitational Athletics Me...,27 - 29 May,2024,International,29 May/24,NaT,NaT,nan,nan,400m Hurdles
1690,Jun Jie Calvin Quek,51.33,None,None,2,None,400m Hurdles,None,None,None,...,Second Belt and Road Invitational Athletics Me...,27 - 29 May,2024,International,29 May/24,NaT,NaT,nan,nan,400m Hurdles
24681,Jun Jie Calvin Quek,52.62,None,None,9,None,400m Hurdles,None,None,None,...,Sydney Track Classic,15-Mar,2025,International,15-Mar-25,NaT,NaT,nan,nan,400m Hurdles


In [4256]:
# Create temporary mapped event column

athletes['MAPPED_EVENT']=''

# Correct javelin category

mask = athletes['EVENT'].str.contains(r'Javelin', na=True)
athletes.loc[mask, 'CATEGORY_EVENT'] = 'Throw'


# Running

#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'50', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'60', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
#mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'80', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'100', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'

mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'200', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'

mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = (athletes['EVENT'].str.contains(r'Dash', na=True) & athletes['DISTANCE'].str.contains(r'400', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'

mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'^600$', na=True, regex=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'800', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1000m'


mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'1500', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'3000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
#mask = athletes['EVENT'].str.contains(r'3000m', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'5000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'^5000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'10000', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'^10\,000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = (athletes['EVENT'].str.contains(r'Run', na=True) & athletes['DISTANCE'].str.contains(r'Mile', na=True))
athletes.loc[mask, 'MAPPED_EVENT'] = '1 Mile'

#mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'



# Hurdles

#mask = athletes['EVENT'].str.contains(r'^80m\sHurdles$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
#mask = athletes['EVENT'].str.contains(r'^80m\shurdles$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
#mask = athletes['EVENT'].str.contains(r'^80\sMeter\sHurdles$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'80', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'

#mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
#mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'


##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
##mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
##athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['DIVISION'].str.contains('None', na=False) & athletes['GENDER'].str.contains(r'Female', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'100', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = (athletes['EVENT'].str.contains(r'100m Hurdles|100m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'



mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.838|0.84', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = ((athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False)) 
         & ((athletes['EVENT_CLASS'].str.contains('None', na=False))|(athletes['EVENT_CLASS']==np.nan)|(athletes['EVENT_CLASS']=='')) 
         & athletes['REGION'].str.contains(r'International', na=False) & (athletes['DIVISION'].str.contains(r'None', na=False)))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'

mask = (athletes['EVENT'].str.contains(r'110m Hurdles|110m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


# Using np.where instead

#athletes['MAPPED_EVENT'] = np.where(((athletes['EVENT']=='110m hurdles|110m Hurdles') & ((athletes['EVENT_CLASS']=='')|athletes['EVENT_CLASS']=='None') & (athletes['REGION']=='International')), '110m Hurdles', ' ')   
                                


mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'1.067', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
#mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'110', na=False) & athletes['EVENT_CLASS'].str.contains(' ', na=True))
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'







#mask = (athletes['EVENT'].str.contains(r'^400m\sHurdles$', na=False) & athletes['EVENT_CLASS'].str.contains(r'', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = ' '
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.84|84cm', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
#mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'^Hurdles$', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['DIVISION'].str.contains(r'Open|Invitational', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False)  & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


#mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
##mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'Hurdles', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762', na=False)& athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles', na=False) & athletes['EVENT_CLASS'].str.contains(r'0.762m', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['EVENT_CLASS'].str.contains('None|0.762|0.914', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = (athletes['EVENT'].str.contains(r'400m Hurdles|400m hurdles', na=False) & athletes['REGION'].str.contains(r'International', na=False))  # this is the correct syntax
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'



# Throws


#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
#mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'600g', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw|Javelin', na=False) & athletes['EVENT_CLASS'].str.contains(r'800g', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = (athletes['EVENT'].str.contains(r'Javelin Throw|Javelin throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False, regex=True) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS']=='4kg'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


#mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Male') & (athletes['EVENT_CLASS'].str.contains(r'7.26', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['GENDER']=='Female') & (athletes['EVENT_CLASS'].str.contains(r'4', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot Put|Shot put', na=False) & (athletes['REGION'].str.contains(r'International', na=False)) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'



#mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
#mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'7.26kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'4.00kg', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = (athletes['EVENT'].str.contains(r'Hammer Throw|Hammer throw', na=False) & (athletes['DIVISION'].str.contains(r'OPEN|Open', na=False)))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'



#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=False))  & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'


#mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=False)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
#mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=False)) & (athletes['GENDER']=='Female'))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'2kg|2.00kg', na=False) & athletes['GENDER'].str.contains(r'Male', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus|Discus throw', na=False) & athletes['EVENT_CLASS'].str.contains(r'1kg|1.00kg', na=False) & athletes['GENDER'].str.contains(r'Female', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

#mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['DIVISION'].str.contains(r'None', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

mask = (athletes['EVENT'].str.contains(r'Discus Throw|Discus throw', na=False) & athletes['REGION'].str.contains(r'International', na=False) & athletes['EVENT_CLASS'].str.contains(r'None', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'^Long\sJump$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

#mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
#mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['EVENT_CLASS'].str.contains(r'0.914', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = (athletes['EVENT'].str.contains(r'Steeplechase', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False)  & athletes['DIVISION'].str.contains(r'OPEN|Open', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'


# Marathon

mask = athletes['EVENT'].str.contains(r'^Marathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\sMarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'
mask = athletes['EVENT'].str.contains(r'^Half\smarathon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half Marathon'


# Walk

#mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'1500', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Race Walk'


#mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'
#mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'3000', na=False))
#athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Race Walk'


#mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
#mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Race Walk'
mask = (athletes['EVENT'].str.contains(r'Race Walk', na=False) & athletes['DISTANCE'].str.contains(r'10000', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'


#mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Race Walk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'400', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'

mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m'
mask = (athletes['EVENT'].str.contains(r'Relay', na=False) & athletes['DISTANCE'].str.contains(r'1600', na=False))
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2435207703.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes['MAPPED_EVENT']=''


In [4257]:
athletes[(athletes['COMPETITION']=='National School Games')]

#athletes[(athletes['EVENT']=='110m Hurdles')].tail(50)

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
1695,Tan Jun Ji,00:15.69,BG,None,102,C,100m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,100m
1696,Madhavan Pranav,01:03.21,BV,None,72,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,400m
1697,"Ko Weijun, Darius",01:00.16,BV,None,50,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,400m
1698,Jeremy Yap Yu Cai,01:02.34,BV,None,68,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,400m
1699,Rachael Lim Zhi Yao,01:20.22,BV,None,52,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,400m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,Teo Lin,00:23.98,SJI(INT),None,12,B,200m,None,None,None,...,National School Games,11-04,2024,Local,04/11/24,NaT,NaT,nan,nan,200m
6152,Faith Jia Yin Koh,01:02.44,SJI(INT),None,3,B,400m,None,None,None,...,National School Games,13-04,2024,Local,04/13/24,NaT,NaT,nan,nan,400m
6153,Caitlin Ng Shan Wen,00:27.35,SJI(INT),None,1,C,200m,None,None,None,...,National School Games,15-04,2024,Local,04/15/24,NaT,NaT,nan,nan,200m
6154,Arjun Mehta,1.55,SJI(INT),None,13,B,High Jump,None,None,None,...,National School Games,19-04,2024,Local,04/19/24,NaT,NaT,nan,nan,High Jump


In [4258]:
athletes[athletes['MAPPED_EVENT']=='Marathon']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
548,"HE, HENRY YONG",2:44:28,Singapore,41,20,None,Marathon,None,None,None,...,Asian Marathon,21-Jan,2024,International,21-Jan-24,NaT,NaT,nan,nan,Marathon
665,DEVATHAS SATIANATHAN?,2:56:03,INDIVIDUAL,-,None,None,Marathon,None,None,None,...,Boston Marathon (USA),15-Apr,2024,International,15-Apr-24,NaT,NaT,nan,nan,Marathon
666,"HIA, CALEB",2:43:40,INDIVIDUAL,-,190,None,Marathon,None,None,None,...,Chicago Marathon (USA),13-Oct,2024,International,13-Oct-24,NaT,NaT,nan,nan,Marathon
668,"TAN, YONG JIN, KERWIN",2:49:39,INDIVIDUAL,-,None,None,Marathon,None,None,None,...,Chicago Marathon (USA),13-Oct,2024,International,13-Oct-24,NaT,NaT,nan,nan,Marathon
669,ANSGAR CHENG,2:51:45,INDIVIDUAL,58,None,None,Marathon,None,None,None,...,Berlin Marathon (Germany),29-Sep,2024,International,29-Sep-24,NaT,NaT,nan,nan,Marathon
670,PU YE,3:22:38,INDIVIDUAL,54,None,None,Marathon,None,None,None,...,Tokyo Marathon (Japan),3-Mar,2024,International,3-Mar-24,NaT,NaT,nan,nan,Marathon
671,ALAN YUMING CAO,2:50:35,INDIVIDUAL,48,None,None,Marathon,None,None,None,...,Wuxi Marathon (China),24-Mar,2024,International,24-Mar-24,NaT,NaT,nan,nan,Marathon
672,CHENG HEOW WONG,2:58:53,INDIVIDUAL,45,None,None,Marathon,None,None,None,...,Seoul Marathon (Korea),17-Mar,2024,International,17-Mar-24,NaT,NaT,nan,nan,Marathon
673,YVONNE ELIZABETH CHEE,3:12:19,INDIVIDUAL,45,None,None,Marathon,None,None,None,...,NN Marathon Rotterdam (Netherlands),14-Apr,2024,International,14-Apr-24,NaT,NaT,nan,nan,Marathon
674,CHEE YONG ONG,2:56:16,INDIVIDUAL,44,None,None,Marathon,None,None,None,...,Seoul Marathon (Korea),17-Mar,2024,International,17-Mar-24,NaT,NaT,nan,nan,Marathon


In [4259]:
for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].astype(str)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:6: SettingWithCopyWarning: 
A 

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.strip()
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2203170124.py:5: SettingWithCopyWarning: 


In [4260]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


athletes.to_csv('athletes_post_map_prod.csv', sep=',', encoding='utf-8-sig', index=False)


In [4261]:
#credentials = service_account.Credentials.from_service_account_file(
#    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
#)

#sql="""
#SELECT NAME, RESULT, RANK, EVENT, CATEGORY_EVENT, GENDER, COMPETITION, STAGE
#FROM `saa-analytics.results.saa_full`
#WHERE STAGE='Final' AND COMPETITION='SEA Game AND RANK='3'
#"""

#benchmarks = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)




In [4263]:
import pandas_gbq
from google.oauth2 import service_account


credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
)

sql="""
SELECT YEAR, EVENT, SUB_EVENT, GENDER, NAME, RESULT, RANK, CATEGORY_EVENT, COMPETITION, STAGE, HEAT
FROM `saa-analytics.benchmarks.saa_benchmarks_prod`
WHERE YEAR='2023' AND COMPETITION='Southeast Asian Games' AND (RANK='3' OR RANK='3.0')
"""

SEAG = pandas_gbq.read_gbq(sql, project_id="saa-analytics", credentials=credentials)



Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [4266]:
SEAG

,YEAR,EVENT,SUB_EVENT,GENDER,NAME,RESULT,RANK,CATEGORY_EVENT,COMPETITION,STAGE,HEAT
0,2023,4 x 100m,None,Male,"{0: '\xa0', 1: ' Jonathan Nyepa, Khairul Hafiz...",39.36,3,Relay,Southeast Asian Games,None,None
1,2023,4 x 400m,None,Male,"{0: '\xa0', 1: ' Muhammad Firdaus Bin Mohamad ...",03:08.8,3,Relay,Southeast Asian Games,None,None
2,2023,4 x 100m,None,Female,"{0: '\xa0', 1: ' Azreen Nabila Alias, Nur Afri...",44.58,3,Relay,Southeast Asian Games,None,None
3,2023,4 x 400m,None,Female,"{0: '\xa0', 1: ' Sukanya Janchaona, Benny Nont...",03:39.3,3,Relay,Southeast Asian Games,None,None
4,2023,400m,None,Male,Frederick Ramirez,46.63,3,Sprint,Southeast Asian Games,None,None
...,...,...,...,...,...,...,...,...,...,...,...
75,2023,100m Hurdles,None,Female,Natchaya Chowpakpang,14.23,3,Hurdles,Southeast Asian Games,Heats,Heat 2
76,2023,Marathon,None,Male,Nguyen Thanh Hoang,2:35:49,3.0,Marathon,Southeast Asian Games,None,None
77,2023,Marathon,None,Female,Christine Organiza Hallasgo,2:50:27,3.0,Marathon,Southeast Asian Games,None,None
78,2023,20km Race Walk,None,Female,Kotchaphon Tangsrivong,1:57:11,3.0,20km Race Walk,Southeast Asian Games,None,None


In [4267]:
SEAG_filtered=SEAG[SEAG['HEAT'].isnull() & SEAG['SUB_EVENT'].isnull()]

In [4268]:
benchmarks=SEAG_filtered

In [4269]:
benchmarks

,YEAR,EVENT,SUB_EVENT,GENDER,NAME,RESULT,RANK,CATEGORY_EVENT,COMPETITION,STAGE,HEAT
0,2023,4 x 100m,None,Male,"{0: '\xa0', 1: ' Jonathan Nyepa, Khairul Hafiz...",39.36,3,Relay,Southeast Asian Games,None,None
1,2023,4 x 400m,None,Male,"{0: '\xa0', 1: ' Muhammad Firdaus Bin Mohamad ...",03:08.8,3,Relay,Southeast Asian Games,None,None
2,2023,4 x 100m,None,Female,"{0: '\xa0', 1: ' Azreen Nabila Alias, Nur Afri...",44.58,3,Relay,Southeast Asian Games,None,None
3,2023,4 x 400m,None,Female,"{0: '\xa0', 1: ' Sukanya Janchaona, Benny Nont...",03:39.3,3,Relay,Southeast Asian Games,None,None
4,2023,400m,None,Male,Frederick Ramirez,46.63,3,Sprint,Southeast Asian Games,None,None
5,2023,"10,000m",None,Male,Than Htike Soe,31:25.5,3,Long,Southeast Asian Games,None,None
6,2023,3000m Steeplechase,None,Male,Pandu Sukarya,08:55.0,3,Steeple,Southeast Asian Games,None,None
7,2023,110m Hurdles,None,Male,John Cabang,13.85,3,Hurdles,Southeast Asian Games,None,None
8,2023,400m Hurdles,None,Male,Jun Jie Calvin Quek,50.75,3,Hurdles,Southeast Asian Games,None,None
9,2023,High Jump,None,Male,Farrell Glenn Felix,2.15,3,Jump,Southeast Asian Games,None,None


In [4270]:
benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/127349024.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmarks.rename(columns = {'RESULT':'BENCHMARK'}, inplace = True)


In [4271]:
benchmarks.drop(['YEAR', 'HEAT', 'NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/2795758766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmarks.drop(['YEAR', 'HEAT', 'NAME', 'RANK', 'CATEGORY_EVENT', 'COMPETITION', 'STAGE'], axis=1, inplace=True)


In [4272]:
benchmarks=benchmarks.reset_index(drop=True)

In [4273]:
# Converts any time format into seconds

def convert_time(i, string, metric):

    global output
    
    l=['discus', 'throw', 'jump', 'vault', 'shot']
        
    string=string.lower()
    
   # print('metric', metric)
    
    try:
        
        if 'w' in metric:  # skip marks with illegal wind speeds
            
        #    print('W', metric)
            
            output=''
            
        else:
            
    
            if any(s in string for s in l)==True:
            
                if 'm' in metric:
            
                    metric=metric.replace('m', '')
                    output=float(str(metric))
            
                elif 'GR' in metric:
            
                    metric=metric.replace('GR', '')
                    output=float(str(metric))
                
                
                else:
    
                    output=float(str(metric))
        
            elif string=='':   # no event description at all!
                
                output='' # return nothing
            
                
        
            else:
        
                searchstring = ":"
                searchstring2 = "."
                substring=str(metric)
                count = substring.count(searchstring)
                count2 = substring.count(searchstring2)
            
                if count==0:
                
                    output=float(substring)
            
            
      #      elif 'marathon' in string and count==2:
                
      #          print('HERE', metric)
                
      #          metric = metric.replace(":", ".", 2)
                
      #          print('METRIC', metric)
                
       #         h,m,s = metric.split(':')            

       #         output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())

                elif '10,000m' in string and count==2:  # fix erroneous timing format from XX:XX:XX to XX:XX.XX
                
                
                    idx = 5 # 6th character position
                    replacement = "."
                    metric = metric[:idx] + replacement + metric[idx+1:]                
                
                    m,s = metric.split(':')            

                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())

        
        
             
                elif (type(metric)==datetime.time or type(metric)==datetime.datetime):
                
                                                
                    time=str(metric)
                    h, m ,s = time.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
            
                                
                elif (count==1 and count2==1):
            
                    m,s = metric.split(':')
                    output = float(datetime.timedelta(minutes=int(m),seconds=float(s)).total_seconds())
                     
                elif (count==1 and count2==2):
                
            
                    metric = metric.replace(".", ":", 1)
            
                    h,m,s = metric.split(':')            
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
                
        
                elif (count==2 and count2==0):
                
            
                    h,m,s = metric.split(':')
                    output = float(datetime.timedelta(hours=int(h),minutes=int(m),seconds=float(s)).total_seconds())
  
            

    except:
        
        pass
                
    return output

In [4274]:
def process_benchmarks(df):
    
    for i in range(len(df)):

        rowIndex = df.index[i]

        input_string=df.iloc[rowIndex,0]
    
        metric=df.iloc[rowIndex,3]
    
        if metric==None:
        
            continue
        
        out = convert_time(i, input_string, metric)
        
        print(rowIndex, input_string, out)

    
        df.loc[rowIndex, 'Metric'] = out
    
    return df

In [4275]:
process_benchmarks(benchmarks)

0 4 x 100m 39.36
1 4 x 400m 188.8
2 4 x 100m 44.58
3 4 x 400m 219.3
4 400m 46.63
5 10,000m 1885.5
6 3000m Steeplechase 535.0
7 110m Hurdles 13.85
8 400m Hurdles 50.75
9 High Jump 2.15
10 Shot Put 17.3
11 400m 53.84
12 1500m 266.3
13 10,000m 2131.0
14 400m Hurdles 59.09
15 Triple Jump 13.46
16 200m 21.02
17 800m 113.9
18 5000m 883.4
19 Discus Throw 50.02
20 Javelin Throw 66.2
21 Decathlon 6891.0
22 800m 129.2
23 5000m 1033.6
24 3000m Steeplechase 660.9
25 High Jump 1.73
26 Discus Throw 45.08
27 Heptathlon 5253.0
28 100m 10.45
29 100m 10.45
30 1500m 239.4
31 Pole Vault 5.2
32 Long Jump 7.62
33 Triple Jump 15.7
34 Hammer Throw 59.76
35 100m 11.75
36 200m 23.6
37 100m Hurdles 13.59
38 Pole Vault 4.0
39 Long Jump 6.02
40 Shot Put 14.44
41 Hammer Throw 49.61
42 Javelin Throw 48.31
43 Marathon 9349.0
44 Marathon 10227.0
45 20km Race Walk 7031.0
46 20km Race Walk 6579.0


,EVENT,SUB_EVENT,GENDER,BENCHMARK,Metric
0,4 x 100m,None,Male,39.36,39.36
1,4 x 400m,None,Male,03:08.8,188.80
2,4 x 100m,None,Female,44.58,44.58
3,4 x 400m,None,Female,03:39.3,219.30
4,400m,None,Male,46.63,46.63
5,"10,000m",None,Male,31:25.5,1885.50
6,3000m Steeplechase,None,Male,08:55.0,535.00
7,110m Hurdles,None,Male,13.85,13.85
8,400m Hurdles,None,Male,50.75,50.75
9,High Jump,None,Male,2.15,2.15


In [4276]:
for i in range(len(benchmarks)):
        
    rowIndex = benchmarks.index[i]

    input_string=benchmarks.iloc[rowIndex,0]
    
    metric=benchmarks.iloc[rowIndex,3]
    
    if metric==None:
        continue
        
    out = convert_time(i, input_string, metric)
    
    print(rowIndex, input_string, out)
     
    benchmarks.loc[rowIndex, 'Metric'] = out

0 4 x 100m 39.36
1 4 x 400m 188.8
2 4 x 100m 44.58
3 4 x 400m 219.3
4 400m 46.63
5 10,000m 1885.5
6 3000m Steeplechase 535.0
7 110m Hurdles 13.85
8 400m Hurdles 50.75
9 High Jump 2.15
10 Shot Put 17.3
11 400m 53.84
12 1500m 266.3
13 10,000m 2131.0
14 400m Hurdles 59.09
15 Triple Jump 13.46
16 200m 21.02
17 800m 113.9
18 5000m 883.4
19 Discus Throw 50.02
20 Javelin Throw 66.2
21 Decathlon 6891.0
22 800m 129.2
23 5000m 1033.6
24 3000m Steeplechase 660.9
25 High Jump 1.73
26 Discus Throw 45.08
27 Heptathlon 5253.0
28 100m 10.45
29 100m 10.45
30 1500m 239.4
31 Pole Vault 5.2
32 Long Jump 7.62
33 Triple Jump 15.7
34 Hammer Throw 59.76
35 100m 11.75
36 200m 23.6
37 100m Hurdles 13.59
38 Pole Vault 4.0
39 Long Jump 6.02
40 Shot Put 14.44
41 Hammer Throw 49.61
42 Javelin Throw 48.31
43 Marathon 9349.0
44 Marathon 10227.0
45 20km Race Walk 7031.0
46 20km Race Walk 6579.0


In [4277]:
benchmarks

,EVENT,SUB_EVENT,GENDER,BENCHMARK,Metric
0,4 x 100m,None,Male,39.36,39.36
1,4 x 400m,None,Male,03:08.8,188.80
2,4 x 100m,None,Female,44.58,44.58
3,4 x 400m,None,Female,03:39.3,219.30
4,400m,None,Male,46.63,46.63
5,"10,000m",None,Male,31:25.5,1885.50
6,3000m Steeplechase,None,Male,08:55.0,535.00
7,110m Hurdles,None,Male,13.85,13.85
8,400m Hurdles,None,Male,50.75,50.75
9,High Jump,None,Male,2.15,2.15


In [4278]:
mask = benchmarks['EVENT'].str.contains(r'jump|throw|Pole|put|Jump|Throw|pole|Put|Decathlon|Heptathlon', na=True)

benchmarks.loc[mask, '2%']=benchmarks['Metric']*0.98
benchmarks.loc[mask, '3.5%']=benchmarks['Metric']*0.965
benchmarks.loc[mask, '5%']=benchmarks['Metric']*0.95

benchmarks.loc[~mask, '2%']=benchmarks['Metric']*1.02
benchmarks.loc[~mask, '3.5%']=benchmarks['Metric']*1.035
benchmarks.loc[~mask, '5%']=benchmarks['Metric']*1.05


#benchmarks.iloc[5, [1]]='10000m run'
#benchmarks.iloc[28, [1]]='10000m run'
#benchmarks.iloc[26, [1]]='1500m'


In [4279]:
benchmarks['MAPPED_EVENT']=benchmarks['EVENT'].str.strip()

In [4280]:
for col in benchmarks.columns:
    benchmarks[col] = benchmarks[col].astype(str)
    benchmarks[col] = benchmarks[col].str.replace('\xa0', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('\r', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.replace('\n', ' ', regex=True)
    benchmarks[col] = benchmarks[col].str.strip()


In [4281]:
benchmarks.head(50)

,EVENT,SUB_EVENT,GENDER,BENCHMARK,Metric,2%,3.5%,5%,MAPPED_EVENT
0,4 x 100m,None,Male,39.36,39.36,40.1472,40.73759999999999,41.328,4 x 100m
1,4 x 400m,None,Male,03:08.8,188.8,192.57600000000002,195.408,198.24,4 x 400m
2,4 x 100m,None,Female,44.58,44.58,45.4716,46.140299999999996,46.809,4 x 100m
3,4 x 400m,None,Female,03:39.3,219.3,223.686,226.97549999999998,230.26500000000001,4 x 400m
4,400m,None,Male,46.63,46.63,47.5626,48.26205,48.96150000000001,400m
5,"10,000m",None,Male,31:25.5,1885.5,1923.21,1951.4924999999998,1979.775,"10,000m"
6,3000m Steeplechase,None,Male,08:55.0,535.0,545.7,553.7249999999999,561.75,3000m Steeplechase
7,110m Hurdles,None,Male,13.85,13.85,14.127,14.334749999999998,14.5425,110m Hurdles
8,400m Hurdles,None,Male,50.75,50.75,51.765,52.52625,53.2875,400m Hurdles
9,High Jump,None,Male,2.15,2.15,2.1069999999999998,2.07475,2.0425,High Jump


In [4282]:
athletes

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,11/10/24,NaT,NaT,nan,nan,Hammer Throw
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,54th SA Inter Club Championships 2024,11/9 to 11/10,2024,Local,11/10/24,NaT,NaT,nan,nan,Pole Vault
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,Pocari Sweat Run 2.4km 2024,9/7 to 9/7,2024,Local,9/7/24,NaT,NaT,nan,nan,
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,6/22/24,NaT,NaT,nan,nan,
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,SMTFA Track & Field Championship 2024,6/22 to 6/22,2024,Local,6/22/24,NaT,NaT,nan,nan,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26004,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,nan,nan,4 x 100m
26005,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,nan,nan,4 x 100m
26006,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,nan,nan,4 x 100m
26007,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,SA Allcomers Meet 2,2/23 to 3/2,2025,Local,3/2/25,NaT,NaT,nan,nan,4 x 100m


In [4283]:
athletes[athletes['COMPETITION']=='National School Games']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,COMPETITION,DATE,YEAR,REGION,event_date,event_date_dt,delta_time,delta_time_conv,event_month,MAPPED_EVENT
1695,Tan Jun Ji,00:15.69,BG,None,102,C,100m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,100m
1696,Madhavan Pranav,01:03.21,BV,None,72,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,400m
1697,"Ko Weijun, Darius",01:00.16,BV,None,50,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,400m
1698,Jeremy Yap Yu Cai,01:02.34,BV,None,68,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,400m
1699,Rachael Lim Zhi Yao,01:20.22,BV,None,52,B,400m,None,None,None,...,National School Games,07-04,2024,Local,04/07/24,NaT,NaT,nan,nan,400m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6151,Teo Lin,00:23.98,SJI(INT),None,12,B,200m,None,None,None,...,National School Games,11-04,2024,Local,04/11/24,NaT,NaT,nan,nan,200m
6152,Faith Jia Yin Koh,01:02.44,SJI(INT),None,3,B,400m,None,None,None,...,National School Games,13-04,2024,Local,04/13/24,NaT,NaT,nan,nan,400m
6153,Caitlin Ng Shan Wen,00:27.35,SJI(INT),None,1,C,200m,None,None,None,...,National School Games,15-04,2024,Local,04/15/24,NaT,NaT,nan,nan,200m
6154,Arjun Mehta,1.55,SJI(INT),None,13,B,High Jump,None,None,None,...,National School Games,19-04,2024,Local,04/19/24,NaT,NaT,nan,nan,High Jump


In [4284]:
# There is a problem with RESULTS column being changed after this statement

#df = athletes.reset_index().merge(benchmarks.reset_index(), on=['MAPPED_EVENT','GENDER'], how='left')
#df = athletes.merge(benchmarks, on=['EVENT','GENDER'], how='left')


In [4285]:
# Merge benchmarks onto athletes on MAPPED_EVENT and GENDER

df = pd.merge(
    left=athletes, 
    right=benchmarks,
    how='left',
    left_on=['MAPPED_EVENT', 'GENDER'],
    right_on=['MAPPED_EVENT', 'GENDER'],
)

In [4286]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,nan,nan,Hammer Throw,Hammer Throw,None,59.76,59.76,58.5648,57.6684,56.772
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,nan,nan,Pole Vault,Pole Vault,None,5.2,5.2,5.096,5.018,4.9399999999999995
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,nan,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,nan,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,nan,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28189,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
28190,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
28191,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
28192,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328


In [4287]:
df[df['NAME']=='Caleb Hia']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
1692,Caleb Hia,2:30:15,None,None,193,None,Marathon,None,None,None,...,nan,nan,Marathon,Marathon,None,2:35:49,9349.0,9535.98,9676.215,9816.45


In [4288]:
df[df['MAPPED_EVENT']=='Decathlon']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
1601,Lucas Le Cong Fun,6431,None,None,1,U20,Decathlon,None,None,None,...,nan,nan,Decathlon,Decathlon,None,6891,6891.0,6753.18,6649.815,6546.45
26861,Jayden Ng,5716,None,None,3,None,Decathlon U18,None,None,None,...,nan,nan,Decathlon,Decathlon,None,6891,6891.0,6753.18,6649.815,6546.45


In [4289]:
# replace '-' with NaN

df['RESULT'] = df['RESULT'].replace(regex=r'–', value=np.NaN)
#df['SEED'] = df['SEED'].replace(regex=r'–', value=np.NaN)


In [4290]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,nan,nan,Hammer Throw,Hammer Throw,None,59.76,59.76,58.5648,57.6684,56.772
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,nan,nan,Pole Vault,Pole Vault,None,5.2,5.2,5.096,5.018,4.9399999999999995
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,nan,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,nan,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,nan,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28189,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
28190,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
28191,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328
28192,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,nan,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328


In [4291]:
df[df['NAME']=='Caleb Hia']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,delta_time_conv,event_month,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%
1692,Caleb Hia,2:30:15,None,None,193,None,Marathon,None,None,None,...,nan,nan,Marathon,Marathon,None,2:35:49,9349.0,9535.98,9676.215,9816.45


In [4292]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


df.to_csv('df_prod.csv', sep=',', encoding='utf-8-sig', index=False)


In [4293]:
# Convert results and seed into seconds format

df.reset_index(drop=True, inplace=True)


for i in range(len(df)):
    
    result_out=''
    
        
    rowIndex = df.index[i]

    input_string=df.iloc[rowIndex,24]    # event description
    
    metric=df.iloc[rowIndex,1] # result
 #   metric_seed=df.iloc[rowIndex, 1]  #seed
    
    if metric=='—' or metric=='DQ' or metric=='SCR' or metric=='FS' or metric=='DNQ' or metric=='DNS' or metric=='NH' or metric=='NM' or metric=='FOUL' or metric=='DNF' or metric=='SR' :
        continue
    
    result_out = convert_time(i, input_string, metric)
    
    #print('line', i, input_string, metric, result_out)

    
#    seed_out = convert_time(i, input_string, metric_seed)
         
    df.loc[rowIndex, 'RESULT_CONV'] = result_out
#    df.loc[rowIndex, 'SEED_CONV'] = seed_out


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/443644260.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[rowIndex, 'RESULT_CONV'] = result_out


In [4294]:
#df["AGE"].fillna(0, inplace=True)
#df['AGE'] = df['AGE'].astype('float')

In [4295]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,event_month,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%,RESULT_CONV
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,nan,Hammer Throw,Hammer Throw,None,59.76,59.76,58.5648,57.6684,56.772,32.4
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,nan,Pole Vault,Pole Vault,None,5.2,5.2,5.096,5.018,4.9399999999999995,4.41
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,nan,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28189,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06
28190,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06
28191,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06
28192,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,nan,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06


In [4296]:
# Choose SEED if better than RESULT

#condition1=df['SEED_CONV']>df['RESULT_CONV']
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


#df['RESULT_BEST']=df['SEED_CONV'].where((condition1 & condition2)|(condition3 & condition4), df['RESULT_CONV'].values)

df['RESULT_BEST'] = df['RESULT_CONV']

In [4297]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%,RESULT_CONV,RESULT_BEST
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,Hammer Throw,Hammer Throw,None,59.76,59.76,58.5648,57.6684,56.772,32.4,32.4
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,Pole Vault,Pole Vault,None,5.2,5.2,5.096,5.018,4.9399999999999995,4.41,4.41
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28189,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06,42.06
28190,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06,42.06
28191,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06,42.06
28192,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,4 x 100m,4 x 100m,None,39.36,39.36,40.1472,40.73759999999999,41.328,42.06,42.06


In [4298]:
df[df['COMPETITION']=='National School Games']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,MAPPED_EVENT,EVENT_y,SUB_EVENT,BENCHMARK,Metric,2%,3.5%,5%,RESULT_CONV,RESULT_BEST
1912,Tan Jun Ji,00:15.69,BG,None,102,C,100m,None,None,None,...,100m,100m,None,10.45,10.45,10.658999999999999,10.815749999999998,10.9725,15.69,15.69
1913,Tan Jun Ji,00:15.69,BG,None,102,C,100m,None,None,None,...,100m,100m,None,10.45,10.45,10.658999999999999,10.815749999999998,10.9725,15.69,15.69
1914,Madhavan Pranav,01:03.21,BV,None,72,B,400m,None,None,None,...,400m,400m,None,46.63,46.63,47.5626,48.26205,48.96150000000001,63.21,63.21
1915,"Ko Weijun, Darius",01:00.16,BV,None,50,B,400m,None,None,None,...,400m,400m,None,46.63,46.63,47.5626,48.26205,48.96150000000001,60.16,60.16
1916,Jeremy Yap Yu Cai,01:02.34,BV,None,68,B,400m,None,None,None,...,400m,400m,None,46.63,46.63,47.5626,48.26205,48.96150000000001,62.34,62.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6709,Teo Lin,00:23.98,SJI(INT),None,12,B,200m,None,None,None,...,200m,200m,None,21.02,21.02,21.4404,21.755699999999997,22.071,23.98,23.98
6710,Faith Jia Yin Koh,01:02.44,SJI(INT),None,3,B,400m,None,None,None,...,400m,400m,None,53.84,53.84,54.9168,55.724399999999996,56.532000000000004,62.44,62.44
6711,Caitlin Ng Shan Wen,00:27.35,SJI(INT),None,1,C,200m,None,None,None,...,200m,200m,None,23.6,23.6,24.072000000000003,24.426,24.78,27.35,27.35
6712,Arjun Mehta,1.55,SJI(INT),None,13,B,High Jump,None,None,None,...,High Jump,High Jump,None,2.15,2.15,2.1069999999999998,2.07475,2.0425,1.55,1.55


In [4299]:
# Change to numeric

df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']] = df[['2%', '3.5%', '5%', 'RESULT_BEST', 'Metric']].apply(pd.to_numeric)

In [4300]:
mask = df['CATEGORY_EVENT'].str.contains(r'Jump|Throw|jump|throw|Decathlon|Heptathlon|decathlon|hetathlon', na=True)

df.loc[mask, 'Delta2'] = df['RESULT_BEST']-df['2%']
df.loc[mask, 'Delta3.5'] = df['RESULT_BEST']-df['3.5%']
df.loc[mask, 'Delta5'] = df['RESULT_BEST']-df['5%']
df.loc[mask, 'Delta_Benchmark'] = df['RESULT_BEST']-df['Metric']

df.loc[~mask, 'Delta2'] =  df['2%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta3.5'] = df['3.5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta5'] = df['5%'] - df['RESULT_BEST']
df.loc[~mask, 'Delta_Benchmark'] = df['Metric'] - df['RESULT_BEST']

#rslt_df['Delta2']=rslt_df['2pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta35']=rslt_df['35pc']-rslt_df['RESULT_CONV']
#rslt_df['Delta5']=rslt_df['5pc']-rslt_df['RESULT_CONV']
df=df.loc[df['COMPETITION']!='Southeast Asian Games'] # Do not include results from SEAG in dataset

In [4301]:
# Performance metric to filter out athletes

df['PERF_SCALAR']=df['Delta5']/df['Metric']*100

In [4302]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


df.to_csv('df_test_prod.csv', sep=',', encoding='utf-8-sig', index=False)


In [4303]:
df[df['MAPPED_EVENT']=='10,000m']

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
540,"Goh, Shaun",32:34.2,Club ZOOM,27,1,Open,Run,10000,,nan,...,NaN,NaN,NaN,1954.2,1954.2,NaN,NaN,NaN,NaN,NaN
604,DARREN CHONG SHUN HAO,37:54:00,SINGAPORE,31,25,None,10000m,10000m,None,None,...,1923.21,1951.4925,1979.775,2274.0,2274.0,-350.79,-322.5075,-294.225,-388.5,-15.604614
605,LEE VANESSA YING ZHUANG,35:55:00,SINGAPORE,26,8,None,10000m,None,None,None,...,2173.62,2205.5850,2237.550,2155.0,2155.0,18.62,50.5850,82.550,-24.0,3.873768
606,VANESSA LEE YING ZHUANG,40:21:00,SINGAPORE,26,10,None,10000m,None,None,None,...,2173.62,2205.5850,2237.550,2421.0,2421.0,-247.38,-215.4150,-183.450,-290.0,-8.608634
611,"GOH, SHAUN",33:48:00,Singapore,27,87,None,10000m,None,None,None,...,1923.21,1951.4925,1979.775,2028.0,2028.0,-104.79,-76.5075,-48.225,-142.5,-2.557677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24884,"Ahmed, Nawaz",39:49.8,Lacticbuds,22,11,Open,Run,10000,None,N034I03,...,1923.21,1951.4925,1979.775,2389.8,2389.8,-466.59,-438.3075,-410.025,-504.3,-21.746221
24885,"Ko, Keane",33:51.9,Club ZOOM,25,1,Open,Run,10000,None,K326A00,...,1923.21,1951.4925,1979.775,2031.9,2031.9,-108.69,-80.4075,-52.125,-146.4,-2.764519
24922,"NG, YEW CHEO",48:27.8,Cougars Athletic Association,39,2,Open,Run,10000,None,Y971Z86,...,2173.62,2205.5850,2237.550,2907.8,2907.8,-734.18,-702.2150,-670.250,-776.8,-31.452370
24923,"Teoh, Katelyn",49:09.4,National University Singapore,20,3,Open,Run,10000,None,K831W05,...,2173.62,2205.5850,2237.550,2949.4,2949.4,-775.78,-743.8150,-711.850,-818.4,-33.404505


In [4304]:
df

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,58.5648,57.6684,56.772,32.4,32.40,-26.1648,-25.2684,-24.372,-27.36,-40.783133
1,"Koh, Wei Shien",4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,5.0960,5.0180,4.940,4.41,4.41,-0.6860,-0.6080,-0.530,-0.79,-10.192308
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28189,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
28190,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
28191,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
28192,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756


In [4305]:
# Read name variations from GCS name lists bucket (Still in beta)


df['NAME'] = df['NAME'].str.replace('\xa0', '', regex=True)
df['NAME'] = df['NAME'].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\r', '', regex=True)
df['NAME'] = df['NAME'].str.replace('\n', '', regex=True)
df['NAME'] = df['NAME'].str.strip()


# Read csv from GCS bucket

file_path = "gs://name_variations/name_variations.csv"
names = pd.read_csv(file_path,
                 sep=",",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})

# Iterate over dataframe and replace names

for index, row in names.iterrows():
        
    df['NAME'] = df['NAME'].replace(regex=rf"{row['VARIATION']}", value=f"{row['NAME']}")


In [4306]:
df_names=df[((df['NAME']=='Heng, Richard')|(df['NAME']=='Marican, Shohib'))]

In [4307]:
df_names.to_csv("names.csv", encoding='utf-8')

In [4308]:
# Exclude foreigners from MALAYSIA, THAILAND etc.

#df_select = df[(df['TEAM']!='Malaysia') & (df['TEAM']!='THAILAND') & (df['TEAM']!='China') & (df['TEAM']!='South Korea') & (df['TEAM']!='Laos') & (df['TEAM']!='Philippines') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Chinese Taipei') & (df['TEAM']!='Gurkha Contingent') & (df['TEAM']!='Australia') & (df['TEAM']!='Piboonbumpen Thailand') & (df['TEAM']!='Hong Kong') & (df['TEAM']!='PERAK')] 

df_select = df[(df['TEAM']!='Malaysia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='China') 
                       &(df['TEAM']!='South Korea')&(df['TEAM']!='Laos') 
                       &(df['TEAM']!='Philippines')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Chinese Taipei')&(df['TEAM']!='Gurkha Contingent') 
                       &(df['TEAM']!='Australia')&(df['TEAM']!='Piboonbumpen Thailand') 
                       &(df['TEAM']!='Hong Kong')&(df['TEAM']!='PERAK')&(df['TEAM']!='Sri Lanka') 
                       &(df['TEAM']!='Indonesia')&(df['TEAM']!='THAILAND')&(df['TEAM']!='MALAYSIA') 
                       &(df['TEAM']!='PHILIPPINES') & (df['TEAM']!='SOUTH KOREA')&(df['TEAM']!='Waseda') 
                       &(df['TEAM']!='LAOS')&(df['TEAM']!='CHINESE TAIPEI')
                       &(df['TEAM']!='INDIA')&(df['TEAM']!='Hong Kong, China')&(df['TEAM']!='AIC JAPAN')] 

In [4309]:
df_select

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,58.5648,57.6684,56.772,32.4,32.40,-26.1648,-25.2684,-24.372,-27.36,-40.783133
1,KOH WEI SHIEN,4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,5.0960,5.0180,4.940,4.41,4.41,-0.6860,-0.6080,-0.530,-0.79,-10.192308
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28189,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
28190,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
28191,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
28192,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756


In [4310]:
df_select

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,58.5648,57.6684,56.772,32.4,32.40,-26.1648,-25.2684,-24.372,-27.36,-40.783133
1,KOH WEI SHIEN,4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,5.0960,5.0180,4.940,4.41,4.41,-0.6860,-0.6080,-0.530,-0.79,-10.192308
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28189,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
28190,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
28191,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
28192,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756


In [4311]:
# Read list of foreigners from GCS bucket

file_path = "gs://name_lists/List of Foreigners.csv"
foreigners = pd.read_csv(file_path,
                 sep=",",
                 encoding="unicode escape",
                 storage_options={"token": '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json'})


In [4312]:
foreigners

,LAST_NAME,FIRST_NAME
0,Aaryan,Greuter Christoph
1,Akahodani,Takayuki
2,Apondar,Audric
3,Brooks,Ruby
4,Brouwer,Cees
...,...,...
236,ISMAIL,MUHAMMAD ZULFIQAR
237,Jayaganeson,Kirtisha
238,LIN,Yu Sian
239,King,Leonard


In [4313]:
foreigners['V1'] = foreigners['LAST_NAME']+' '+foreigners['FIRST_NAME']
foreigners['V2'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']
foreigners['V3'] = foreigners['LAST_NAME']+', '+foreigners['FIRST_NAME']
foreigners['V4'] = foreigners['FIRST_NAME']+' '+foreigners['LAST_NAME']

for1 = foreigners['V1'].dropna().tolist()
for2 = foreigners['V2'].dropna().tolist()
for3 = foreigners['V3'].dropna().tolist()
for4 = foreigners['V4'].dropna().tolist()

foreign_list = for1+for2+for3+for4 

foreign_list_casefold=[s.casefold() for s in foreign_list]

exclusions = foreign_list_casefold

no_foreigners_list = df_select.loc[~df['NAME'].str.casefold().isin(exclusions)]  # ~ means NOT IN. DROP spex carded athletes

In [4314]:
no_foreigners_list

,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,UNIQUE_ID,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,"Chong Bin Mohd Isham, Akid",32.40m,FAC,34,2,Open,Hammer Throw,0,(7.26kg),A512E90,...,58.5648,57.6684,56.772,32.4,32.40,-26.1648,-25.2684,-24.372,-27.36,-40.783133
1,KOH WEI SHIEN,4.41m,NUS,29,3,Open,Pole Vault,0,,L435B06,...,5.0960,5.0180,4.940,4.41,4.41,-0.6860,-0.6080,-0.530,-0.79,-10.192308
2,"Wen Qiang, Koh",07:38.0,SIT,27,4,Tertiary Men,Run,2400,,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
3,"Harinee, Sivakumaru",15.03,SKM,4,7,5-Jun,Run,60,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
4,"Din, Muhamad Edir",22.43m,SMA,50,5,Masters,Javelin Throw,0,,nan,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28189,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,23,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
28190,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
28191,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,15,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756
28192,"{'1339': 'BIN MUHAMMAD RASHID, EMIR', '1340': ...",42.06,Singapore,16,2,Open,Relay,400,Trial,None,...,40.1472,40.7376,41.328,42.06,42.06,-1.9128,-1.3224,-0.732,-2.70,-1.859756


In [4315]:
# Choose the best performing event for each athlete

#top_performers_clean = excluded_list.sort_values(['NAME','PERF_SCALAR'],ascending=False).groupby('NAME').head(1) # Choose top performing event per NAME


In [4316]:
# Choose the best result for each event participated by every athlete

top_performers_clean = no_foreigners_list.sort_values(['MAPPED_EVENT', 'NAME','PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)


In [4317]:
top_performers_clean.reset_index(inplace=True)


In [4318]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


top_performers_clean.to_csv('top_performers_clean_prod.csv', encoding='utf-8')

In [4319]:
top_performers_clean

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,18613,"yeo, husni",10.09m,Singapore Polytechnic,19,17,Open,Triple Jump,0,,...,15.3860,15.1505,14.915,10.09,10.09,-5.2960,-5.0605,-4.825,-5.61,-30.732484
1,17687,"ooi, chen si",9.22m,Ngee Ann Polytechnic,18,9,Open,Triple Jump,0,,...,13.1908,12.9889,12.787,9.22,9.22,-3.9708,-3.7689,-3.567,-4.24,-26.500743
2,16180,"chua, keira",10.16m,Cedar Girls Secondary School,16,5,U18,Triple Jump,0,,...,13.1908,12.9889,12.787,10.16,10.16,-3.0308,-2.8289,-2.627,-3.30,-19.517088
3,27475,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,15.3860,15.1505,14.915,10.47,10.47,-4.9160,-4.6805,-4.445,-5.23,-28.312102
4,23950,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,15.3860,15.1505,14.915,11.67,11.67,-3.7160,-3.4805,-3.245,-4.03,-20.668790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16533,8794,"., Mohd Farrel",31.28,Seng Kang Primary School,10,38,U11,Dash,150,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
16534,13819,"., Dharkshitha",12:46.5,Cedar Girls Secondary School,14,5,U15,Race Walk,1500,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
16535,12514,"., Cayden",06:05.1,North Vista,14,54,Open,Mile Run,1,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
16536,7333,"., Asher",11.76,Erovra Club,6,16,7-Aug,Dash,60,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [4320]:
# Choose best performance for each event

#tiered_performers = top_performers_clean.sort_values(['GENDER', 'MAPPED_EVENT', 'PERF_SCALAR'],ascending=False).groupby(['MAPPED_EVENT', 'NAME']).head(1)

tiered_performers = top_performers_clean


In [4321]:
tiered_performers

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,2%,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR
0,18613,"yeo, husni",10.09m,Singapore Polytechnic,19,17,Open,Triple Jump,0,,...,15.3860,15.1505,14.915,10.09,10.09,-5.2960,-5.0605,-4.825,-5.61,-30.732484
1,17687,"ooi, chen si",9.22m,Ngee Ann Polytechnic,18,9,Open,Triple Jump,0,,...,13.1908,12.9889,12.787,9.22,9.22,-3.9708,-3.7689,-3.567,-4.24,-26.500743
2,16180,"chua, keira",10.16m,Cedar Girls Secondary School,16,5,U18,Triple Jump,0,,...,13.1908,12.9889,12.787,10.16,10.16,-3.0308,-2.8289,-2.627,-3.30,-19.517088
3,27475,"Zheng, Justin De",10.47m,National Junior College,14,12,U15,Triple Jump,0,None,...,15.3860,15.1505,14.915,10.47,10.47,-4.9160,-4.6805,-4.445,-5.23,-28.312102
4,23950,"Zhao, Daniel",11.67m,Hwa Chong Institution,14,3,U15,Triple Jump,0,None,...,15.3860,15.1505,14.915,11.67,11.67,-3.7160,-3.4805,-3.245,-4.03,-20.668790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16533,8794,"., Mohd Farrel",31.28,Seng Kang Primary School,10,38,U11,Dash,150,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
16534,13819,"., Dharkshitha",12:46.5,Cedar Girls Secondary School,14,5,U15,Race Walk,1500,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
16535,12514,"., Cayden",06:05.1,North Vista,14,54,Open,Mile Run,1,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
16536,7333,"., Asher",11.76,Erovra Club,6,16,7-Aug,Dash,60,,...,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [4322]:
# Identify Tier 1/2/3 performers

#top_performers_clean['TIER'] = np.where((top_performers_clean['Delta_Benchmark']>=0), 'Tier 1',    
#                                np.where(((top_performers_clean['Delta_Benchmark']<0) & (top_performers_clean['Delta2']>=0)), 'Tier2',
#                                np.where(((top_performers_clean['Delta2']<0) & (top_performers_clean['Delta3.5']>=0)), 'Tier3', ' ')))


tiered_performers['TIER'] = np.where((tiered_performers['Delta_Benchmark']>=0), 'Tier 1',    
                                np.where(((tiered_performers['Delta_Benchmark']<0) & (tiered_performers['Delta2']>=0)), 'Tier 2',
                                np.where(((tiered_performers['Delta2']<0) & (tiered_performers['Delta3.5']>=0)), 'Tier 3', 
                                np.where(((tiered_performers['Delta3.5']<0) & (tiered_performers['Delta5']>=0)), 'Tier 4', ' '))))



In [4323]:
tiered_performers[tiered_performers['COMPETITION']=='National School Games']

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,3.5%,5%,RESULT_CONV,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER
8,5679,Yvette Lee,10.13,VJC,None,7,A,Triple Jump,None,None,...,12.9889,12.787,10.13,10.13,-3.0608,-2.8589,-2.657,-3.33,-19.739970,
10,2102,Yuan Yinze,11.54,CH,None,15,B,Triple Jump,None,None,...,15.1505,14.915,11.54,11.54,-3.8460,-3.6105,-3.375,-4.16,-21.496815,
11,4942,Yong Zeheng,9.96,SJI,None,15,C,Triple Jump,None,None,...,15.1505,14.915,9.96,9.96,-5.4260,-5.1905,-4.955,-5.74,-31.560510,
13,5902,"Yeow Kai Xin, Charis",9.26,ACJC,None,17,A,Triple Jump,None,None,...,12.9889,12.787,9.26,9.26,-3.9308,-3.7289,-3.527,-4.20,-26.203566,
17,4521,Yen Ming Zhen,11.93,NJC,None,9,B,Triple Jump,None,None,...,15.1505,14.915,11.93,11.93,-3.4560,-3.2205,-2.985,-3.77,-19.012739,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16482,2656,Adrian Tan Buo Yuan,09:28.59,RI,None,17,C,1500m Race Walk,None,None,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,
16483,2297,Aden Seah Jia Kai,00:29.87,HG,None,12,C,200m Hurdles,None,0.762m,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,
16489,4631,Abdul Hadi Bin Ahmad Abdillah,12:43.23,NJC,None,46,B,3000m,None,None,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,
16493,5238,Aarush Vikram,00:15.60,SPS,None,3,C,100m Hurdles,None,0.840m,...,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,


In [4324]:
# Drop rows without a SEAG benchmark

final_df = tiered_performers[tiered_performers['BENCHMARK'].notna()]


In [4325]:
# Process dates to extract age

# Map NSG divisions into age

mask = (final_df['DIVISION'].str.contains(r'A', na=False))
final_df.loc[mask, 'AGE'] = '18.5'

mask = (final_df['DIVISION'].str.contains(r'B', na=False))
final_df.loc[mask, 'AGE'] = '16'

mask = (final_df['DIVISION'].str.contains(r'C', na=False))
final_df.loc[mask, 'AGE'] = '13.5'

mask = (final_df['DIVISION'].str.contains(r'O', na=False))
final_df.loc[mask, 'AGE'] = '12'



In [4326]:
def length(string):

    B = ''
    year = ''

    try:

        length = len(string)

        if length == 2:

            string = '19' + string

        elif length == 1:

            string = ''

        else:

            pass

        if string is not None or len(string) != 1:

            B = parser.parse(string, dayfirst=True)
                        
    except:

        pass

    return B


final_df['DOB_new'] = final_df['DOB'].apply(length)



#B = parser.parse("10-09-2021", dayfirst = True)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/3934348418.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['DOB_new'] = final_df['DOB'].apply(length)


In [4327]:
final_df['DOB_new'] = pd.to_datetime(final_df['DOB_new'], errors='coerce')

final_df['year_extract']=final_df['DOB_new'].dt.strftime('%Y')

final_df['year_extract'] = pd.to_numeric(final_df['year_extract'])

final_df['age_extract'] = 2025 - final_df['year_extract']


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1594838337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['DOB_new'] = pd.to_datetime(final_df['DOB_new'], errors='coerce')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1594838337.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['year_extract']=final_df['DOB_new'].dt.strftime('%Y')
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1594838337.py:5: SettingWithCopyWarning: 
A value is 

In [4328]:
def age(number):  # correct negative age numbers
    
    if number<0:
        
        number+=100
        
    return number


final_df['age_extract']=final_df['age_extract'].apply(age)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1795310803.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['age_extract']=final_df['age_extract'].apply(age)


In [4329]:
# If NSG event then choose AGE otherwise choose age_extract

condition1 = final_df['COMPETITION']=='National School Games'
#condition2=((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))
#condition3=df['SEED_CONV']<df['RESULT_CONV']
#condition4=~((df['CATEGORY_EVENT']=='Jump')|(df['CATEGORY_EVENT']=='Throw'))


final_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/4177143290.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['age_extract'] = final_df['AGE'].where((condition1), final_df['age_extract'].values)


In [4330]:
# Change to numeric

final_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_22644/1263148356.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[['age_extract']] = final_df[['age_extract']].apply(pd.to_numeric)


In [4331]:
final_df[final_df['COMPETITION']=='National School Games']

,index,NAME,RESULT,TEAM,AGE,COMPETITION_RANK,DIVISION,EVENT_x,DISTANCE,EVENT_CLASS,...,RESULT_BEST,Delta2,Delta3.5,Delta5,Delta_Benchmark,PERF_SCALAR,TIER,DOB_new,year_extract,age_extract
8,5679,Yvette Lee,10.13,VJC,18.5,7,A,Triple Jump,None,None,...,10.13,-3.0608,-2.85890,-2.6570,-3.33,-19.739970,,NaT,NaN,18.5
10,2102,Yuan Yinze,11.54,CH,16,15,B,Triple Jump,None,None,...,11.54,-3.8460,-3.61050,-3.3750,-4.16,-21.496815,,NaT,NaN,16.0
11,4942,Yong Zeheng,9.96,SJI,13.5,15,C,Triple Jump,None,None,...,9.96,-5.4260,-5.19050,-4.9550,-5.74,-31.560510,,NaT,NaN,13.5
13,5902,"Yeow Kai Xin, Charis",9.26,ACJC,18.5,17,A,Triple Jump,None,None,...,9.26,-3.9308,-3.72890,-3.5270,-4.20,-26.203566,,NaT,NaN,18.5
17,4521,Yen Ming Zhen,11.93,NJC,16,9,B,Triple Jump,None,None,...,11.93,-3.4560,-3.22050,-2.9850,-3.77,-19.012739,,NaT,NaN,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12136,3585,Ahmad Syukri Bin Mohd Fazli,00:15.35,CRS,13.5,97,C,100m,None,None,...,15.35,-4.6910,-4.53425,-4.3775,-4.90,-41.889952,,NaT,NaN,13.5
12138,2975,Adrian Tang,00:13.42,US,13.5,45,C,100m,None,None,...,13.42,-2.7610,-2.60425,-2.4475,-2.97,-23.421053,,NaT,NaN,13.5
12144,5219,Aarnav Inampudi,00:13.51,SPS,13.5,51,C,100m,None,None,...,13.51,-2.8510,-2.69425,-2.5375,-3.06,-24.282297,,NaT,NaN,13.5
12170,4739,AAqil Aimar Fazly,00:15.07,PLS,13.5,95,C,100m,None,None,...,15.07,-4.4110,-4.25425,-4.0975,-4.62,-39.210526,,NaT,NaN,13.5


In [4332]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/OCTC/')


final_df.to_csv('tiered_performers_prod.csv', encoding='utf-8')

In [4333]:
# Rank everyone for published ranking lists

published_ranking = final_df.sort_values(['MAPPED_EVENT','GENDER','PERF_SCALAR'], ascending=[False, False, False])
published_ranking['Rank'] = published_ranking.groupby(['GENDER', 'MAPPED_EVENT']).cumcount() + 1

published_ranking.to_csv('published_ranking_prod.csv', encoding='utf-8')

In [4242]:
# Convert time format for marathon and 5000m into mm:ss.00
# Choose the correct column indices or you will get erratic timings

import datetime

#s=247.779

#datetime.datetime.fromtimestamp(s).strftime('%M:%S.%f')

all_ranking=all_ranking.reset_index(drop=True)


#all_ranking[['2%', '3.5%', '5%']] = df[['2%', '3.5%', '5%']].apply(pd.to_numeric)


#all_ranking['2%'] = all_ranking['2%'].astype("string")
#all_ranking['3.5%'] = all_ranking['3.5%'].astype("string")
#all_ranking['5%'] = all_ranking['5%'].astype("string")


for i in range(len(all_ranking)):
        
    rowIndex = all_ranking.index[i]

    event=all_ranking.iloc[rowIndex,21]
        
    
    time_base2=all_ranking.iloc[rowIndex,25]
    time_base3=all_ranking.iloc[rowIndex,26]
    time_base5=all_ranking.iloc[rowIndex,27]
    
        
    if metric==None:
        continue
        
    if event=='800m' or event=='10,000m' or event=='5000m' or event=='3000m Steeplechase' or event=='1500m':
        
      #  print(i, event, time_base2, time_base3, time_base5)

        
        

            
        
        date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
        date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
        date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)
        
    #    print(date_preconvert2, date_preconvert3, date_preconvert5)
            
        
        output2 = datetime.datetime.strftime(date_preconvert2, "%M:%S.%f")
        output3 = datetime.datetime.strftime(date_preconvert3, "%M:%S.%f")
        output5 = datetime.datetime.strftime(date_preconvert5, "%M:%S.%f")
            
     #   print(event, output2, output3, output5)

                    
       #     top_performers_clean.loc[rowIndex, '2%_timing'] = output2
       #     top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
       #     top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            
   
        all_ranking.at[rowIndex, '2%'] = output2 # copy over time format
        all_ranking.at[rowIndex, '3.5%'] = output3
        all_ranking.at[rowIndex, '5%'] = output5


            


        
    elif event=='Marathon':
        
      #  print(time_base2, time_base3, time_base5)

        
        try:
            

        
            date_preconvert2 = datetime.datetime.utcfromtimestamp(time_base2)
            date_preconvert3 = datetime.datetime.utcfromtimestamp(time_base3)
            date_preconvert5 = datetime.datetime.utcfromtimestamp(time_base5)

            
            
            output2 = datetime.datetime.strftime(date_preconvert2, "%H:%M:%S")
            output3 = datetime.datetime.strftime(date_preconvert3, "%H:%M:%S")
            output5 = datetime.datetime.strftime(date_preconvert5, "%H:%M:%S")

            
        
        #    top_performers_clean.loc[rowIndex, '2%_timing'] = output2
        #    top_performers_clean.loc[rowIndex, '3.5%_timing'] = output3
        #    top_performers_clean.loc[rowIndex, '5%_timing'] = output5
            
            all_ranking.at[rowIndex, '2%'] = output2 # copy over time format
            all_ranking.at[rowIndex, '3.5%'] = output3
            all_ranking.at[rowIndex, '5%'] = output5

            
         #   print('output', output2, output3, output5)


        
        except:
            
            pass
                        
             
